## 0. Description
- 우선 기능 단위로 만들고 하나의 모듈(클래스)로 만들어서 작동시켜보자

## 1. Load Library

In [51]:
import os
import sys
import pickle
import numpy as np
import subprocess
from tqdm import tqdm

from safetensors.torch import load_file

## 2. Convert safetensors -> pkl

In [48]:
BASE_PATH = "/Users/dataeng/modelscan-test/modelscan/unscaned/3901296/request-01/model"

ext_in = 'safetensors'
ext_out = 'npy'

MODEL_NAME = "Mistral-Nemo-Instruct-2407"

In [52]:
def load_model(ext_in:str,path:str):
    tensors = None

    if ext_in == 'safetensors':
        print("execute load_file")
        tensors = load_file(path)
    
    return tensors

def store_model(ext_out:str,path:str,tensors=None):
    '''
    tensor는 어떤 library를 사용하여 load 하는지에 따라 형식이 달라질 수 있다.
    추후 다양한 type을 지원할 수 있도록 모듈을 생성해야할 것이다.

    1. torch : collections.OrderedDict
    '''
    if ext_out == 'pkl':
        # pickle을 사용하여 state_dict 저장
        with open(path, 'wb') as f:
            pickle.dump(tensors, f)

def convert(base_path:str,model_name:str,file_name:str,ext_in:str, ext_out:str):
    # 1. load model weight with ext_in extension file
    load_model_file_path = os.path.join(base_path,ext_in,model_name,file_name)
    print(f"Source Model File Path : {load_model_file_path}")

    # 각 확장자 별로 load함수를 달리 해야한다.
    tensors = load_model(ext_in=ext_in,path=load_model_file_path)

    # 2. store model weight with ext_out extension file
    # .pkl 파일 경로 지정
    store_model_file_name = '.'.join([file_name.split('.')[0],ext_out])
    store_model_file_path = os.path.join(base_path,ext_out,model_name,store_model_file_name)
    print(f"Store File Model File Path : {store_model_file_path}")
    store_model(ext_out=ext_out,path=store_model_file_path,tensors=tensors)  

# def model_scan(base_path:str,model_name:str,file_name:str,ext_in:str, ext_out:str):
    

## Summary

| Approach | Popularity | Risk of Model Serialization Attack Exploitability |
| --- | --- | --- |
| Pickle Variants | Very high | Very high |
| Tensorflow SavedModel | High | Medium |
| H5 (Keras) | High | Low (except Keras Lambda layer) |
| Inference Only | Medium | Low |
| Vector/Tensor Only | Low | Very low |

In [ ]:
class ModelScan():
    def __init__(self,base_path:str,model_name:str,ext_in:str,ext_out:str):
        self.base_path:str = base_path
        self.model_name:str = model_name
        self.ext_in:str = ext_in
        self.ext_out:str = ext_out
        self.tensors:dict = {}

    def __model_file_list(self,isLoad=True) -> list:
        '''
        load model file list
        '''
        model_weight_file_list = None

        # true -> load, false -> store
        extension = ext_in if isLoad else ext_out

        model_file_list = os.listdir(os.path.join(self.base_path,extension,self.model_name))
        model_weight_file_list = [fn for fn in model_file_list if fn.endswith('.'+extension) and not 'consolidated' in fn]
    
        return model_weight_file_list
    
    def __load_model(self,path:str):
        '''
        다양한 extension input을 지원할 수 있도록 해야함.
        '''
        tensor = None

        if self.ext_in == 'safetensors':
            print("execute load_file")
            tensor = load_file(path)
        
        return tensor
    
    def __store_model(self,path:str,tensors=None):
        '''
        tensor는 어떤 library를 사용하여 load 하는지에 따라 형식이 달라질 수 있다.
        추후 다양한 type을 지원할 수 있도록 모듈을 생성해야할 것이다.

        1. torch : collections.OrderedDict
        '''
        if self.ext_out == 'pkl':
            # pickle을 사용하여 state_dict 저장
            with open(path, 'wb') as f:
                pickle.dump(tensors, f)
        elif self.ext_out == 'npy':
            for name, array in tensors.items():
                np.save(f'{name}.npy', array)

    def __convert(self):
        '''
        description:
            Scaning All Model files(like .safetensor,,)
            이 convert 함수를 멀티 thread 혹은 processing으로 구현하면 좋을 것 같은데...
        return:

        '''
        # 1. load model weight with ext_in extension file
        load_model_file_list = self.__model_file_list(isLoad=True)
        for f in load_model_file_list:
            load_model_file_path = os.path.join(self.base_path,self.ext_in,self.model_name,f)
            print(f"Source Model File Path : {load_model_file_path}")

            # 각 확장자 별로 load함수를 달리 해야한다.
            model_weight_file_name = f.split('.')[0]
            self.tensors[model_weight_file_name] = self.__load_model(path=load_model_file_path)

        # 2. store model weight with ext_out extension file
        # .pkl 파일 경로 지정
        for i,f in enumerate(load_model_file_list):
            store_model_file_name = f.split('.')[0]
            store_model_file_path = os.path.join(self.base_path,ext_out,self.model_name,store_model_file_name+'.'+self.ext_out)
            print(f"Store File Model File Path : {store_model_file_path}")
            self.__store_model(path=store_model_file_path,tensors=self.tensors[store_model_file_name]) 
    
    def scan(self):
        # convert
        # check해야 할 필요가 없는 파일은 pass 하도록 구현
        # convert가 필요한 확장자들을 미리 정의해놓자.
        self.__convert()
        
        # 명령 실행
        store_model_folder_path = os.path.join(self.base_path,ext_out,self.model_name)

        command = f"modelscan -p {store_model_folder_path} -r json -o output-file-name.json"
        commands = command.split(" ")
        result = subprocess.run(commands, capture_output=True, text=True)

        # 결과 출력
        # print("Return Code:", result.returncode)
        # print("Standard Output:", result.stdout)
        # print("Standard Error:", result.stderr)

In [65]:
BASE_PATH = "/Users/dataeng/modelscan-test/modelscan/unscaned/3901296/request-01/model"

ext_in = 'safetensors'
ext_out = 'pkl'

MODEL_NAME = "Mistral-Nemo-Instruct-2407"


if __name__=="__main__":
    scanner = ModelScan(
                base_path=BASE_PATH,
                model_name=MODEL_NAME,
                ext_in=ext_in,
                ext_out=ext_out
            )

    scanner.scan()

Source Model File Path : /Users/dataeng/modelscan-test/modelscan/unscaned/3901296/request-01/model/safetensors/Mistral-Nemo-Instruct-2407/model-00003-of-00005.safetensors
execute load_file
Source Model File Path : /Users/dataeng/modelscan-test/modelscan/unscaned/3901296/request-01/model/safetensors/Mistral-Nemo-Instruct-2407/model-00002-of-00005.safetensors
execute load_file
Source Model File Path : /Users/dataeng/modelscan-test/modelscan/unscaned/3901296/request-01/model/safetensors/Mistral-Nemo-Instruct-2407/model-00001-of-00005.safetensors
execute load_file
Source Model File Path : /Users/dataeng/modelscan-test/modelscan/unscaned/3901296/request-01/model/safetensors/Mistral-Nemo-Instruct-2407/model-00004-of-00005.safetensors
execute load_file
Source Model File Path : /Users/dataeng/modelscan-test/modelscan/unscaned/3901296/request-01/model/safetensors/Mistral-Nemo-Instruct-2407/model-00005-of-00005.safetensors
execute load_file
Store File Model File Path : /Users/dataeng/modelscan-t

## 3. Huggingface 모델 다운로드 로직 구성
- 시나리오는 다음과 같다.
    1. 모델 파일 요청(특정 user, 특정 모델)
    2. 모델 파일 탐색
    3. 모델 파일 취약성 점검 여부 확인(protect AI Knowledge Base or Huggingface)
    4. token 발급 or 기존 데이터 사용
    5. 다운로드 to blob(blob설계 필요. azcopy 특성을 고려한)
    6. p-region copy 수행(MSP의 허가가 필요)

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="bert-base-uncased")
